# Vertex AI: Knowledge Base Setup with Vector Search + Firestore
**Author:** Zain Khan  
**Created:** 2025-05-05 03:41

This notebook sets up a full pipeline for embedding documents, storing them in Firestore, and querying them with Vertex AI Matching Engine.

In [ ]:
# Step 1: Environment Setup
project_id = "chatgpt-418315"
location = "us-central1"
index_endpoint_id = "3898149151489130496"
deployed_index_id = "kbv1"

In [ ]:
# Step 2: Initialize GenAI Client
from google.cloud import aiplatform
from vertexai.preview.generative_models import GenerativeModel
import vertexai

vertexai.init(project=project_id, location=location)
aiplatform.init(project=project_id, location=location)

In [ ]:
# Step 3: Create text embedding
from vertexai.preview.language_models import TextEmbeddingModel
embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko")

def get_text_embedding(text: str):
    embeddings = embedding_model.get_embeddings([text])
    return embeddings[0].values

In [ ]:
# Step 4: Upload test doc to Firestore
from google.cloud import firestore
import uuid

fs_client = firestore.Client()
sample_text = "LLMs are transforming support with generative AI. This is a test doc."
doc_id = f"sample-doc-{uuid.uuid4().hex[:8]}"
fs_doc = fs_client.collection("documents").document(doc_id)
fs_doc.set({"pages": {0: sample_text}})
print(f"✅ Uploaded Firestore doc {doc_id}")

In [ ]:
# Step 5: Query Matching Engine
endpoint = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_id)
query_vector = get_text_embedding("What is generative AI?")
matches = endpoint.find_neighbors(
    deployed_index_id=deployed_index_id,
    queries=[query_vector],
    num_neighbors=1
)
print(matches)